In [ ]:
def find_biggest_rest_station(geodataframe):
    """
    Returns the biggest rest station.

    Parameters:
    geodataframe (geopandas geodataframe): contains polygon/multipolygon geometries

    Returns:
    The id of the biggest rest station.
    """

    def polygon_to_bbox(polygon):
        """
        Transforms a polygon into a square bounding box.

        Parameters:
        polygon (Polygon): input polygon

        Returns:
        The resulting square bounding box bounds as a list: [left, bottom, right, top].
        """


        # Calculate the bounding box of the polygon
        minx, miny, maxx, maxy = polygon.bounds
        bbox = [minx, miny, maxx, maxy]

        return bbox

    # Initialize variables
    max_span_overall = 0
    biggest_rest_station = None
    biggest_rest_station_bbox = None
    biggest_rest_station_name = None

    # Loop over all rows in the df, calculate the bbox and find the largest span
    for index, row in geodataframe.iterrows():
        bbox = polygon_to_bbox(row["geometry"])
        minx, miny, maxx, maxy = bbox
        x_span = maxx - minx
        y_span = maxy - miny

        max_span = max(x_span, y_span)

        if max_span > max_span_overall:
            max_span_overall = max_span
            biggest_rest_station = row["id_rest"]
            biggest_rest_station_bbox = [minx, miny, maxx, maxy]
            biggest_rest_station_name = row["name"]

    print(f"The biggest rest station is {biggest_rest_station_name} (id: {biggest_rest_station}) with a maximum bounding box span of {max_span_overall}")

    return biggest_rest_station

In [ ]:
def calculate_bbox_dimensions(polygon):
    """
    Calculates the dimensions of the bounding box of a polygon in degrees.

    Parameters:
    polygon (Polygon): input polygon

    Returns:
    Width and height of the bounding box in degrees.
    """
    # Get the bounds of the polygon
    minx, miny, maxx, maxy = polygon.bounds
    # Calculate width and height in degrees
    width = maxx - minx
    height = maxy - miny
    return width, height

In [ ]:
rest_station_id = find_biggest_rest_station(rest_stations_ver)

In [ ]:
rest_station_id_height = rest_stations_ver[rest_stations_ver["id_rest"] == "lon_7.0388979_lat_52.3124962"]["geometry"].values[0]
print(rest_station_id_height)

In [ ]:
def find_needed_image_pixel_size(polygon, WMS_resolution = 0.2):
    minx, miny, maxx, maxy = polygon.bounds

    print("The bounding box has the following coordinates", [minx, miny, maxx, maxy])

    x_line = shapely.LineString([(minx, miny), (maxx, miny)])
    y_line = shapely.LineString([(minx, miny), (minx, maxy)])

    geod = Geod(ellps="WGS84")

    length_x = math.ceil(geod.geometry_length(x_line))
    length_y = math.ceil(geod.geometry_length(y_line))

    print(f"The bounding box has a width of: {length_x}m and a height of: {length_y}m")

    max_length = max(length_x, length_y)

    needed_resolution = math.ceil(max_length / WMS_resolution)

    print("The needed minimum pixel size is:", needed_resolution, "pixels.")

In [ ]:
find_needed_image_pixel_size(rest_station_id_height)